In [2]:
from PIL import Image
import os
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# set results_folder to the folder in the same directory with the name "results"
results_folder = '../smallresults/'
train_folder = '../smallresults/train/'
test_folder = '../smallresults/test/'
validation_folder = '../smallresults/val/'

In [4]:
# Create a model
model = Sequential()
# Add a convolutional layer with 32 filters, each of size 3x3
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))

# Add convolutional layers 
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

# Output
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [5]:
# Compile the model
model.compile(Adam(learning_rate=0.003), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Create a generator that will take the data and create batches of grayscale images
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_folder, target_size=(150, 150), batch_size=13, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_folder, target_size=(150, 150), batch_size=6, class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(validation_folder, target_size=(150, 150), batch_size=8, class_mode='categorical')

Found 858 images belonging to 2 classes.
Found 186 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


In [9]:
# Train the model
model.fit(train_generator, steps_per_epoch=66, epochs=10, validation_data=validation_generator, validation_steps=23)

Epoch 1/10
66/66 [==============================] - 101s 2s/step - loss: 1.1684 - accuracy: 0.5058 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
66/66 [==============================] - 98s 1s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/10
66/66 [==============================] - 100s 2s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
66/66 [==============================] - 99s 1s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
66/66 [==============================] - 98s 1s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
66/66 [==============================] - 100s 2s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/10
66/66 [==============================] - 96s 1s/step - loss: 0.6933 - accuracy: 0.4744 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/10
66/66 

In [ ]:
# Save the model
model.save('verydeeplearning.h5')